#Text classification
Categorization of different lyrics by genre. The aim of this code is to find out wheter a new lyrics is a 'typical pop song' or not. 

! Prior note: This code has been constructed as a learning exercise.

#Operattionalization
Definition of the goal: find a way to distinguise between pop songs and other genres based on lyrics. 

To do: Find overlap between pop lyrics and (all lyrics - pop lyrics) = distinguisted words.

Methods: I will use the method we discussed in exercise 1.2. In this exercise we subset the words that where in one part of the dataset, but not in the other. We looked into the words that they had in common and the words that set them apart from eachother. To find out whether a song is a typical popsong or not, we need to know what sets them appart from the rest of the lyrics. 

Operalization: 
1. load the data. 
2. preprocess the text using tokenization + lemmatization + word positions (VERB + ADJ + NOUN --> substract the most meaningfull words) and flattening over two subsets (if we don't use tokenization and flattening the counter object will count characters instead of words). One subset contains all the popsong data. The other subset contains all the data - the pop song data. 
3. use the Counter object to find the words that will differ. create a vocabulary containing the counts of both dets. calculate the frequency in which the words appear. Then calculate the log-likelihood and p-value. Sort the results based on the log-likelyhood score in descending order. Evaluate the results using the log-likelihood and p-value. The words with the highest log-likelyhood scores are the most distinctive for popsongs. 
N the more often they appear to be in a popsong, the more influence they have on setting a song appart from other songs. Therefore that word will be more typical for a popsong then other words.  
4. evaluate the new popsong by training a text classification model. Then provide the new lyrics to the model and predict wheter or not it is a popsong. 

Time period:
Starting at 1990, most years afterwards are having more then 100 values in this dataset. Therefore I decided on a time period between 1990 - 2020. I assume this will help me to get a better p-value later on and therefore are more likely to obtain significant results.

In [46]:
#load datasets english cleaned lyrics and index2newdate (so the date will be closer related to the true timeperiod of the song)
#Here I used the code provided with exercise 3.1
import pickle
import pandas as pd

PATH_DF = 'english_cleaned_lyrics.csv' # change this to your dir
PATH_CORRECTION = 'indx2newdate.p' # idem

def load_dataset(data_path, path_correction):
    df = pd.read_csv(data_path)
    indx2newdate = pickle.load(open(PATH_CORRECTION, 'rb'))
    df['year'] = df['index'].apply(lambda x: int(indx2newdate[x][0][:4]) if indx2newdate[x][0] != '' else 0)
    return df[df.year > 1960][['song', 'year', 'artist', 'genre', 'lyrics']]

df = load_dataset(PATH_DF, PATH_CORRECTION)


In [47]:
df

,song,year,artist,genre,lyrics
0,ego-remix,2009,beyonce-knowles,Pop,Oh baby how you doing You know I'm gonna cut r...
5,all-i-could-do-was-cry,2008,beyonce-knowles,Pop,I heard Church bells ringing I heard A choir s...
6,once-in-a-lifetime,2008,beyonce-knowles,Pop,This is just another day that I would spend Wa...
9,why-don-t-you-love-me,2009,beyonce-knowles,Pop,N n now honey You better sit down and look aro...
16,poison,2009,beyonce-knowles,Pop,You're bad for me I clearly get it I don't see...
18,beautiful-liar,2007,beyonce-knowles,Pop,Ay Ay Ay Nobody likes to be played Oh Beyonce ...
19,beautiful-liar-spanish,2007,beyonce-knowles,Pop,Ay Ay Nobody likes being played Ay Oh Beyonc B...
20,beautiful-liar-spanglish-version,2007,beyonce-knowles,Pop,Ay Ay Nobody likes being played Ay Oh Beyonce ...
27,if,2008,beyonce-knowles,Pop,He is always laughin and flirting with me And ...
29,flaws-all,2007,beyonce-knowles,Pop,I'm a train wreck in the morning I'm a bitch i...


In [48]:
#decide on time period
df['year'].value_counts(ascending=False)
df = df[(df.year >= 1990)|(df.year <= 2020)]

In [50]:
#packages
#!pip install spacy
!python -m spacy download en
from collections import Counter
import spacy
import pandas as pd

from tqdm import tqdm_notebook
from scipy.stats import chi2_contingency

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-12-10 12:52:31.170210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-12-10 12:52:31.170606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [51]:
#pre-processing
nlp = spacy.load("en_core_web_sm")
lyrics = df.lyrics.values
processed_texts = [text for text in tqdm_notebook(nlp.pipe(lyrics, 
                                              n_process=-1, 
                                              disable=["ner",
                                                       "parser"]),
                                          total=len(lyrics
                                                   ))]

C:\Users\bbsln\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/160856 [00:00<?, ?it/s]

In [55]:
df['processed_texts'] = processed_texts

In [53]:
flatten = lambda t: [item for sublist in t for item in sublist]

#tokenize and flatten text over categories pop and notpop
#flattening is important when using a Counter later, otherwise it will only count characters
pop = df[(df.genre == 'Pop')].processed_texts
pop = [[word.lemma_.lower() for word in text if word.pos_ == 'VERB' or word.pos == "ADJ" or word.pos_ == "NOUN" or word.pos_ == "PROPN"] for text in pop]
pop = flatten(pop)

notpop = df[(df.genre != 'Pop')].processed_texts
notpop = [[word.lemma_.lower() for word in text if word.pos_ == 'VERB' or word.pos == "ADJ" or word.pos_ == "NOUN" or word.pos_ == "PROPN"] for text in notpop]
notpop = flatten(notpop)

In [56]:
#Here we can see that the data is imbalanced
print(len(pop),len(notpop))

1965428 11709561


In [57]:
#create a label to train the model (pop or notpop)
label = []
for genre in df.genre:
    if genre =='Pop':
        label.append('pop')
    else:
        label.append('notpop')
        
df['label'] = label  

In [58]:
#find distinctive words between two sets of words
def distinctive_words(target_corpus, reference_corpus):
    counts_c1 = Counter(target_corpus) 
    counts_c2 = Counter(reference_corpus)
    vocabulary = set(list(counts_c1.keys()) + list(counts_c2.keys()))
    freq_c1_total = sum(counts_c1.values()) 
    freq_c2_total = sum(counts_c2.values()) 
    results = []
    for word in vocabulary:
        freq_c1 = counts_c1[word]
        freq_c2 = counts_c2[word]
        freq_c1_other = freq_c1_total - freq_c1
        freq_c2_other = freq_c2_total - freq_c2
        llr, p_value,_,_ = chi2_contingency([[freq_c1, freq_c2], 
                      [freq_c1_other, freq_c2_other]],
                      lambda_='log-likelihood') 
        if freq_c2 / freq_c2_other > freq_c1 / freq_c1_other:
            llr = -llr
        result = {'word':word, 
                    'llr':llr,
                    'p_value': p_value}
        results.append(result)
    results_df = pd.DataFrame(results)
    return results_df

In [59]:
results_comp = distinctive_words(pop, notpop)

In [14]:
#distincive words when it comes to pop songs
results_comp.sort_values('llr', ascending=False)

,llr,p_value,word
31526,21928.318640,0.000000e+00,love
21994,9172.515200,0.000000e+00,baby
21406,4425.997336,0.000000e+00,heart
103333,4301.300819,0.000000e+00,feel
1708,4263.665113,0.000000e+00,know
78147,3883.460255,0.000000e+00,let
117588,3691.409754,0.000000e+00,want
90442,3020.222114,0.000000e+00,wanna
39446,3004.540875,0.000000e+00,need
116979,2940.623489,0.000000e+00,girl


Pop songs are very likely to include the words love, baby, halo, christmas and girl. Pop songs are very unlikely to include the words fuck, niggas, bitch, shit and nigga. 

In [60]:
#create a new dataset for the new lyrics 
#source = https://stackoverflow.com/questions/50097501/how-to-open-text-file-in-google-collab?rq=1
import numpy as np
with open('new_lyrics.txt', 'r') as f:

    new_lyrics=f.read()

vocab = sorted(set(new_lyrics))

vocab_to_int = {c: i for i, c in enumerate(vocab)}

int_to_vocab = dict(enumerate(vocab))

encoded = np.array([vocab_to_int[c] for c in new_lyrics], dtype=np.int32)

In [61]:
#clean up the new_lyrics text
def tokenize(text):
    punctuations = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for punctuation in punctuations:
        text = text.replace(punctuation, '')
    text = text.lower() 
    return text

text = new_lyrics
tokenized_texts = [tokenize(text)]
tokenized_texts=[tokenized_texts.replace('\n',"")for tokenized_texts in tokenized_texts] #remove the sign for new line (\n)
tokenized_texts

['the sky breaks open and the rain falls down oh and the pain is blinding but we carry on seems like the end of everything when the one you love turns their back on you and the whole world falls down on you its the end of the world its the end of the world well im holding onbut the world keeps dragging me down its the end of the world its the end of the world well im holding on but the world keeps dragging me down i got my heart on lockdown and my eyes on the lookout but i just know that im never gonna win this they can keep the lights on they can keep the music loud i dont need anything when i got my music and im holding on the sky breaks open and the rain falls down and the pain is blinding but we carry on they tell you lies im holding on they tell you liesthe sky breaks open and the rain falls down oh and the pain is blinding but we carry on seems like the end of everything when the one you love turns their back on you and the whole world falls down on you its the end of the world i

In [62]:
#classify new lyrics (compare through genres in a plot - which is it more a like with?)
from sklearn.model_selection import train_test_split

X = list(df.lyrics.values) # the lyric test --> X
y = list(df.label.values) # I want to predict pop song or notpop song--> Y
labels = ['pop','notpop']

X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer() 

cv.fit(X_train_str) 

X_train = cv.transform(X_train_str)
X_test = cv.transform(X_test_str)

In [63]:
#create the vocabulary for the train and test sets
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer() 

tfidf.fit(X_train_str) 

X_train = tfidf.transform(X_train_str)
X_test = tfidf.transform(X_test_str)

In [82]:
#This is where I tried to oversample the minority class
#!pip install imblearn
from imblearn.over_sampling import SMOTE 
oversample = SMOTE()
X_OS, y_OS = oversample.fit_resample(X_train, y_train)
# summarize the new class distribution
counter = Counter(y_OS)
print("After Running SMOTE", counter)
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y_OS == label)[0]
	plt.scatter(X_OS[row_ix, 0], X_OS[row_ix, 1], label=str(label))
plt.legend()
plt.show()
#equal amount majority and minority class

  Found existing installation: scikit-learn 0.21.2
    Uninstalling scikit-learn-0.21.2:
      Successfully uninstalled scikit-learn-0.21.2


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Toegang geweigerd: 'c:\\users\\bbsln\\anaconda3\\lib\\site-packages\\~klearn\\feature_extraction\\_hashing.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



ModuleNotFoundError: No module named 'imblearn'

In [64]:
#train a logistic regression - model used for text classification
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='sag') #i used sag as i want to handle a bigger dataset
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [66]:
#evaluate the model 
from sklearn.metrics import classification_report

y_pred = lr.predict(X_test)

print(classification_report(y_test, y_pred, 
                          target_names=labels))

C:\Users\bbsln\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


              precision    recall  f1-score   support

         pop       0.86      0.99      0.92     27394
      notpop       0.60      0.07      0.12      4778

    accuracy                           0.85     32172
   macro avg       0.73      0.53      0.52     32172
weighted avg       0.82      0.85      0.80     32172



In [72]:
#see the values of the true positives, false positives, true negatives and false negatives
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

[[27181   213]
 [ 4455   323]]


In [84]:
#the new song
new_example = ['the sky breaks open and the rain falls down oh and the pain is blinding but we carry on seems like the end of everything when the one you love turns their back on you and the whole world falls down on you its the end of the world its the end of the world well im holding onbut the world keeps dragging me down its the end of the world its the end of the world well im holding on but the world keeps dragging me down i got my heart on lockdown and my eyes on the lookout but i just know that im never gonna win this they can keep the lights on they can keep the music loud i dont need anything when i got my music and im holding on the sky breaks open and the rain falls down and the pain is blinding but we carry on they tell you lies im holding on they tell you liesthe sky breaks open and the rain falls down oh and the pain is blinding but we carry on seems like the end of everything when the one you love turns their back on you and the whole world falls down on you its the end of the world its the end of the world well im holding onbut the world keeps dragging me down its the end of the world its the end of the world well im holding on but the world keeps dragging me down i got my heart on lockdown and my eyes on the lookout but i just know that im never gonna win this they can keep the lights on they can keep the music loud i dont need anything when i got my music  and im holding on the sky breaks open and the rain falls down and the pain is blinding but we carry on they tell you lies im holding on they tell you lies']
new_example_bow = cv.transform(new_example)

In [77]:
#predict the class of the new song
prediction_probabilities = lr.predict_proba(new_example_bow)
for i, proba in enumerate(prediction_probabilities):
    probability_pop = proba[0]
    probability_notpop = proba[1]
    print("{}: {}% pop; {}% not-pop".format(new_example[i], 
                                                    round(probability_notpop, 3),
                                                    round(probability_pop, 3)))

the sky breaks open and the rain falls down oh and the pain is blinding but we carry on seems like the end of everything when the one you love turns their back on you and the whole world falls down on you its the end of the world its the end of the world well im holding onbut the world keeps dragging me down its the end of the world its the end of the world well im holding on but the world keeps dragging me down i got my heart on lockdown and my eyes on the lookout but i just know that im never gonna win this they can keep the lights on they can keep the music loud i dont need anything when i got my music and im holding on the sky breaks open and the rain falls down and the pain is blinding but we carry on they tell you lies im holding on they tell you liesthe sky breaks open and the rain falls down oh and the pain is blinding but we carry on seems like the end of everything when the one you love turns their back on you and the whole world falls down on you its the end of the world its

#Is this a typical pop song? Why yes or no?

This model shows that the song is definitly not a pop song. However, my model is imbalenced. Therefore the chance the model predicts the song to be 'notpop' is way higher than the chance that the model predicts the song to be 'pop'. 

Let's take a look at the results of the model:<br>
Precision: these are the true positives over the positive predicted values. Which means that if we take a look at 'predicted as pop' most of these are indeed part of the pop genre. We see that the prediction of 'notpop' has a precision of 0.6. This means that in 60 percent of the cases a lyrics was indeed part of the notpop category. But 40 percent of the time my model was wrong. This implies there is quite some bias in our model. Perhaps this holds a relation with the imbalance in the data.

Recall: 
Recall shows how many of the actual positives are captured by the prediction. So how many of the 'notpop' are predicted to be 'notpop'. We can see here that there is a very low score of only 7 percent captured by the model. The pop song has a high score of 99 percent, but since a lot of 'nonpop' values are predicted as 'pop' there is a lot of bias within the categorization.

Discussion:
To improve the performance of the model I would first try to balance the data by oversampling the minority class. To vectorize the words I choose the Bag Of Words model. I could try to use the spacy for tokenizatior or TF-IDF instead. I could also try a different model, like the BERT model, which is context aware. This would probably help to improve the classification of the song lyrics. 

